In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
credit_df = pd.read_csv('/content/drive/MyDrive/Kaggle_Study/나은/creditcard-user-overdue-prediction/train.csv')
credit_df.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [ ]:
print('\n ### 학습 데이터 정보 ### \n')
print(credit_df.info())


 ### 학습 데이터 정보 ### 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-nul

### 1. 결측치 처리

In [ ]:
print('데이터 세트 Null 값의 개수 \n', credit_df.isnull().sum())

데이터 세트 Null 값의 개수 
 index               0
gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       8171
family_size         0
begin_month         0
credit              0
dtype: int64


In [ ]:
credit_df['occyp_type'].fillna('N', inplace=True)
print('데이터 세트 Null 값의 개수 ', credit_df.isnull().sum().sum())

데이터 세트 Null 값의 개수  0


In [ ]:
print('gender 값 분포 :\n', credit_df['gender'].value_counts())
print('\n\n income_total 값 분포 :\n', credit_df['credit'].value_counts())
print('\n\n occyp_type 값 분포 :\n', credit_df['occyp_type'].value_counts())

gender 값 분포 :
 F    17697
M     8760
Name: gender, dtype: int64


 income_total 값 분포 :
 2.0    16968
1.0     6267
0.0     3222
Name: credit, dtype: int64


 occyp_type 값 분포 :
 N                        8171
Laborers                 4512
Core staff               2646
Sales staff              2539
Managers                 2167
Drivers                  1575
High skill tech staff    1040
Accountants               902
Medicine staff            864
Cooking staff             457
Security staff            424
Cleaning staff            403
Private service staff     243
Low-skill Laborers        127
Waiters/barmen staff      124
Secretaries                97
Realty agents              63
HR staff                   62
IT staff                   41
Name: occyp_type, dtype: int64


### 2. 데이터 확인

In [ ]:
credit_df.groupby(['edu_type', 'credit'])['credit'].count()

edu_type                       credit
Academic degree                0.0           2
                               1.0           7
                               2.0          14
Higher education               0.0         909
                               1.0        1751
                               2.0        4502
Incomplete higher              0.0         114
                               1.0         246
                               2.0         660
Lower secondary                0.0          28
                               1.0          59
                               2.0         170
Secondary / secondary special  0.0        2169
                               1.0        4204
                               2.0       11622
Name: credit, dtype: int64

In [ ]:
credit_df.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,N,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


### 3. 데이터 Encoding

In [ ]:
from sklearn import preprocessing

def encode_features(dataDF, data2DF):
  features = ['income_type', 'edu_type', 'family_type','house_type','occyp_type', 'gender','car','reality']
  enc = preprocessing.OneHotEncoder()
  enc = enc.fit(dataDF[features])
  train_onehot_df = pd.DataFrame(enc.transform(dataDF[features]).toarray(), 
             columns=enc.get_feature_names(features))
  
  data2DF.drop(['family_size'], axis=1, inplace=True)
  data2DF.drop(features, axis=1, inplace=True)
  train = pd.concat([data2DF, train_onehot_df], axis=1)

  return train


credit_df = encode_features(credit_df, credit_df)
credit_df.head(30)

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,begin_month,credit,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_N,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y
0,0,0,202500.0,-13899,-4709,1,0,0,0,-6.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,1,1,247500.0,-11380,-1540,1,0,0,1,-5.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,2,0,450000.0,-19087,-4434,1,0,1,0,-22.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,3,0,202500.0,-15088,-2092,1,0,1,0,-37.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,4,0,157500.0,-15037,-2105,1,0,0,0,-26.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
5,5,2,270000.0,-13413,-4996,1,0,0,1,-18.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
6,6,0,315000.0,-17570,-1978,1,0,0,1,-41.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
7,7,0,270000.0,-14896,-5420,1,0,0,1,-53.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
8,8,1,180000.0,-15131,-1466,1,0,0,1,-38.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
9,9,0,103500.0,-15785,-1308,1,0,0,0,-5.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(data = credit_df.corr(), annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

### 지금까지의 전처리를 함수로 만들기

In [ ]:
def fillna(df):
  
  df['occyp_type'].fillna('N', inplace=True)
  return df

def encode_features(dataDF, data2DF):
  features = ['income_type', 'edu_type', 'family_type','house_type','occyp_type', 'gender','car','reality']
  enc = preprocessing.OneHotEncoder()
  enc = enc.fit(dataDF[features])
  train_onehot_df = pd.DataFrame(enc.transform(dataDF[features]).toarray(), 
             columns=enc.get_feature_names(features))
  
  data2DF.drop(['family_size'], axis=1, inplace=True)
  data2DF.drop(features, axis=1, inplace=True)
  train = pd.concat([data2DF, train_onehot_df], axis=1)

  return train

def transform_features(df):
  df = fillna(df)
  df = encode_features(df,df)
  return df

### 4. 머신러닝 모델 학습해보기

In [ ]:
credit_df = pd.read_csv('/content/drive/MyDrive/Kaggle_Study/나은/creditcard-user-overdue-prediction/train.csv')
y_credit_df = credit_df['credit']
X_credit_df = credit_df.drop('credit', axis=1)

X_credit_df = transform_features(X_credit_df)
print(X_credit_df.head())
print('전체 데이터의 개수 :',X_credit_df.size)

   index  child_num  income_total  ...  car_Y  reality_N  reality_Y
0      0          0      202500.0  ...    0.0        1.0        0.0
1      1          1      247500.0  ...    0.0        0.0        1.0
2      2          0      450000.0  ...    1.0        0.0        1.0
3      3          0      202500.0  ...    0.0        0.0        1.0
4      4          0      157500.0  ...    1.0        0.0        1.0

[5 rows x 56 columns]
전체 데이터의 개수 : 1481592


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_credit_df, y_credit_df, test_size=0.2, random_state=11)

In [ ]:
print('train_test_split 한 후의 train data 개수 :',X_train.size)
print('train_test_split 한 후의 test data 개수 :',X_test.size)

train_test_split 한 후의 train data 개수 : 1185240
train_test_split 한 후의 test data 개수 : 296352


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
xgb_clf = XGBClassifier(n_estimators=1000, random_state=156)

xgb_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="mlogloss", eval_set=[(X_train, y_train), (X_test, y_test)])

[0]	validation_0-mlogloss:1.05463	validation_1-mlogloss:1.05592
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[1]	validation_0-mlogloss:1.01766	validation_1-mlogloss:1.02015
[2]	validation_0-mlogloss:0.986451	validation_1-mlogloss:0.989995
[3]	validation_0-mlogloss:0.959941	validation_1-mlogloss:0.964453
[4]	validation_0-mlogloss:0.937375	validation_1-mlogloss:0.942708
[5]	validation_0-mlogloss:0.918148	validation_1-mlogloss:0.924312
[6]	validation_0-mlogloss:0.901701	validation_1-mlogloss:0.908578
[7]	validation_0-mlogloss:0.887646	validation_1-mlogloss:0.895227
[8]	validation_0-mlogloss:0.875591	validation_1-mlogloss:0.883814
[9]	validation_0-mlogloss:0.865267	validation_1-mlogloss:0.874116
[10]	validation_0-mlogloss:0.856377	validation_1-mlogloss:0.865852
[11]	validation_0-mlogloss:0.848757	validation_1-mlogloss:0.85881
[12]	validation_0-mlogloss:0.842183	validat

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=156,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
xgb_clf = XGBClassifier(n_estimators=1000, random_state=156, max_depth=12, num_leaves=2024, learning_rate=0.1, subsample=0.8, colsample_bytree=0.8, eta=0.05
)

xgb_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="mlogloss", eval_set=[(X_train, y_train), (X_test, y_test)], verbose=10)

[0]	validation_0-mlogloss:1.04785	validation_1-mlogloss:1.05429
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[10]	validation_0-mlogloss:0.777668	validation_1-mlogloss:0.849391
[20]	validation_0-mlogloss:0.670816	validation_1-mlogloss:0.792881
[30]	validation_0-mlogloss:0.605941	validation_1-mlogloss:0.771355
[40]	validation_0-mlogloss:0.558683	validation_1-mlogloss:0.759489
[50]	validation_0-mlogloss:0.520053	validation_1-mlogloss:0.752385
[60]	validation_0-mlogloss:0.483666	validation_1-mlogloss:0.747324
[70]	validation_0-mlogloss:0.453361	validation_1-mlogloss:0.743371
[80]	validation_0-mlogloss:0.422437	validation_1-mlogloss:0.740352
[90]	validation_0-mlogloss:0.39804	validation_1-mlogloss:0.738371
[100]	validation_0-mlogloss:0.375726	validation_1-mlogloss:0.737523
[110]	validation_0-mlogloss:0.355181	validation_1-mlogloss:0.736375
[120]	validation_0-mlogloss:0.

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.05, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, num_leaves=2024, objective='multi:softprob',
              random_state=156, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=0.8, verbosity=1)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
xgb_clf = XGBClassifier(n_estimators=1000, random_state=156, max_depth=12, num_leaves=2024, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, eta=0.05
)

xgb_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="mlogloss", eval_set=[(X_train, y_train), (X_test, y_test)], verbose=10)

[0]	validation_0-mlogloss:1.07275	validation_1-mlogloss:1.07597
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[10]	validation_0-mlogloss:0.892393	validation_1-mlogloss:0.930794
[20]	validation_0-mlogloss:0.789276	validation_1-mlogloss:0.858471
[30]	validation_0-mlogloss:0.721058	validation_1-mlogloss:0.818251
[40]	validation_0-mlogloss:0.670385	validation_1-mlogloss:0.794224
[50]	validation_0-mlogloss:0.637999	validation_1-mlogloss:0.782283
[60]	validation_0-mlogloss:0.605488	validation_1-mlogloss:0.772644
[70]	validation_0-mlogloss:0.580933	validation_1-mlogloss:0.766108
[80]	validation_0-mlogloss:0.560358	validation_1-mlogloss:0.76158
[90]	validation_0-mlogloss:0.539915	validation_1-mlogloss:0.758063
[100]	validation_0-mlogloss:0.524223	validation_1-mlogloss:0.755374
[110]	validation_0-mlogloss:0.507121	validation_1-mlogloss:0.752007
[120]	validation_0-mlogloss:0.

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.05, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=12,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, num_leaves=2024, objective='multi:softprob',
              random_state=156, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=0.8, verbosity=1)

### 5. GridSearch

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimators=1000)

params = {'learning_rate':[0.3,0.001],'max_depth':[5,7], 'min_child_weight':[3,5], 'colsample_bytree':[0.25,0.5,0.75]}

In [ ]:
from sklearn.model_selection import GridSearchCV
gridcv = GridSearchCV(xgb_clf, param_grid=params, cv=3)
gridcv.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="mlogloss", eval_set=[(X_train, y_train), (X_test, y_test)])

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[1301]	validation_0-mlogloss:0.833393	validation_1-mlogloss:0.85766
[1302]	validation_0-mlogloss:0.83331	validation_1-mlogloss:0.857593
[1303]	validation_0-mlogloss:0.833227	validation_1-mlogloss:0.857523
[1304]	validation_0-mlogloss:0.833143	validation_1-mlogloss:0.857455
[1305]	validation_0-mlogloss:0.833063	validation_1-mlogloss:0.857393
[1306]	validation_0-mlogloss:0.832984	validation_1-mlogloss:0.857333
[1307]	validation_0-mlogloss:0.832911	validation_1-mlogloss:0.857277
[1308]	validation_0-mlogloss:0.832829	validation_1-mlogloss:0.857207
[1309]	validation_0-mlogloss:0.832747	validation_1-mlogloss:0.857139
[1310]	validation_0-mlogloss:0.832672	validation_1-mlogloss:0.85708
[1311]	validation_0-mlogloss:0.832588	validation_1-mlogloss:0.857008
[1312]	validation_0-mlogloss:0.832503	validation_1-mlogloss:0.85694
[1313]	validation_0-mlogloss:0.832429	validation_1-mlogloss:0.856884
[1314]	validation_0-mlogloss:0.832361	validation_1-mlogloss:0.856829
[1

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
xgb_clf = XGBClassifier(n_estimators=2000, colsample_bytree=0.5, learning_rate=0.3, max_depth= 7,  min_child_weight=3)

xgb_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="mlogloss", eval_set=[(X_train, y_train), (X_test, y_test)])

[0]	validation_0-mlogloss:0.982548	validation_1-mlogloss:0.990504
Multiple eval metrics have been passed: 'validation_1-mlogloss' will be used for early stopping.

Will train until validation_1-mlogloss hasn't improved in 100 rounds.
[1]	validation_0-mlogloss:0.913518	validation_1-mlogloss:0.928868
[2]	validation_0-mlogloss:0.868746	validation_1-mlogloss:0.888503
[3]	validation_0-mlogloss:0.837467	validation_1-mlogloss:0.86342
[4]	validation_0-mlogloss:0.816785	validation_1-mlogloss:0.846975
[5]	validation_0-mlogloss:0.79645	validation_1-mlogloss:0.831823
[6]	validation_0-mlogloss:0.784342	validation_1-mlogloss:0.824523
[7]	validation_0-mlogloss:0.775513	validation_1-mlogloss:0.818334
[8]	validation_0-mlogloss:0.764893	validation_1-mlogloss:0.811759
[9]	validation_0-mlogloss:0.755933	validation_1-mlogloss:0.807519
[10]	validation_0-mlogloss:0.750112	validation_1-mlogloss:0.805059
[11]	validation_0-mlogloss:0.745585	validation_1-mlogloss:0.802728
[12]	validation_0-mlogloss:0.742524	vali

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.3, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=2000, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

### 6. 테스트 데이터에 적용

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Kaggle_Study/나은/creditcard-user-overdue-prediction/test.csv')
test_df.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
0,26457,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,1,0,1,0,NaN,2.0,-60.0
1,26458,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,1,0,1,0,Core staff,2.0,-36.0
2,26459,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,1,0,Laborers,2.0,-40.0
3,26460,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,1,0,0,Drivers,2.0,-41.0
4,26461,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,1,0,0,Managers,2.0,-8.0


In [ ]:
test_df = transform_features(test_df)
test_df.head()

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,begin_month,income_type_Commercial associate,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Academic degree,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Civil marriage,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Accountants,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_N,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,gender_F,gender_M,car_N,car_Y,reality_N,reality_Y
0,26457,0,112500.0,-21990,365243,1,0,1,0,-60.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,26458,0,135000.0,-18964,-8671,1,0,1,0,-36.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,26459,0,69372.0,-15887,-217,1,1,1,0,-40.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,26460,0,112500.0,-19270,-2531,1,1,0,0,-41.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,26461,0,225000.0,-17822,-9385,1,1,0,0,-8.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
dpredictions = xgb_clf.predict_proba(test_df)

df = pd.DataFrame(dpredictions)
df.rename(columns={0:'0'}, inplace=True)
df

,0,1,2
0,0.014295,0.308721,0.676984
1,0.017133,0.104273,0.878594
2,0.030249,0.197844,0.771907
3,0.025895,0.086528,0.887577
4,0.062929,0.508599,0.428473
...,...,...,...
9995,0.051690,0.212274,0.736037
9996,0.018235,0.330068,0.651697
9997,0.010471,0.026541,0.962988
9998,0.027983,0.153658,0.818360


In [ ]:
test_new_df = pd.read_csv('/content/drive/MyDrive/Kaggle_Study/나은/creditcard-user-overdue-prediction/test.csv')
test_new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          10000 non-null  int64  
 1   gender         10000 non-null  object 
 2   car            10000 non-null  object 
 3   reality        10000 non-null  object 
 4   child_num      10000 non-null  int64  
 5   income_total   10000 non-null  float64
 6   income_type    10000 non-null  object 
 7   edu_type       10000 non-null  object 
 8   family_type    10000 non-null  object 
 9   house_type     10000 non-null  object 
 10  DAYS_BIRTH     10000 non-null  int64  
 11  DAYS_EMPLOYED  10000 non-null  int64  
 12  FLAG_MOBIL     10000 non-null  int64  
 13  work_phone     10000 non-null  int64  
 14  phone          10000 non-null  int64  
 15  email          10000 non-null  int64  
 16  occyp_type     6848 non-null   object 
 17  family_size    10000 non-null  float64
 18  begin_m

In [ ]:
result = pd.concat([test_new_df['index'], df], axis=1)
result

,index,0,1,2
0,26457,0.014295,0.308721,0.676984
1,26458,0.017133,0.104273,0.878594
2,26459,0.030249,0.197844,0.771907
3,26460,0.025895,0.086528,0.887577
4,26461,0.062929,0.508599,0.428473
...,...,...,...,...
9995,36452,0.051690,0.212274,0.736037
9996,36453,0.018235,0.330068,0.651697
9997,36454,0.010471,0.026541,0.962988
9998,36455,0.027983,0.153658,0.818360


In [ ]:
result.to_csv('xgboost_gridSearchCV_feature_encoding.csv', index=False)